In [28]:
import torch
import sys
import numpy as np
import os
import yaml
import matplotlib.pyplot as plt
import torchvision
import pandas as pd
from datetime import datetime
from tqdm import tqdm
from sklearn.metrics import cohen_kappa_score
from torch.utils.tensorboard import SummaryWriter
from sklearn.metrics import accuracy_score, cohen_kappa_score, classification_report, precision_score, recall_score, f1_score, roc_auc_score

In [29]:
from torch.utils.data import DataLoader
import torchvision.transforms as transforms
from torchvision import datasets

In [30]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print("Using device:", device)
a = "/home/hdamari/SSL/MURA-v1.1./MURA-v1.1//"

Using device: cuda


In [31]:
!ls

backup_finetune-Copy1.ipynb  finetuned
checkpoint_0018.pth.tar      finetune.ipynb
colab_test_hand_only.pkl     LICENSE.txt
colab_test.pkl		     models
colab_train_hand_only.pkl    pretraining_training_scripts.ipynb
colab_train.pkl		     __pycache__
data_aug		     README.md
df_test.pkl		     requirements.txt
df_train.pkl		     run.py
env.yml			     runs
ex3.ipynb		     simclr.py
exceptions		     utils.py
feature_eval


In [32]:
from data_aug.contrastive_learning_dataset import MuraDataset

In [33]:
batch_size = 16

In [34]:
transform = transforms.Compose([transforms.Resize((244, 244)), transforms.ToTensor(), transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])])

In [37]:
body_parts = ["XR_ELBOW", "XR_SHOULDER", "XR_HAND"]
weights_no_normalize = ["runs/07-10_18-15-27_Elbow/XR_ELBOW_resnet50checkpoint_0100.pth.tar",
                       "runs/07-10_18-40-29_Shoulder/XR_SHOULDER_resnet50checkpoint_0100.pth.tar",
                       "runs/07-10_19-21-48_Hand/XR_HAND_resnet50checkpoint_0100.pth.tar"]
weights_normalzie = ["runs/07-11_11-25-30_XR_ELBOW_resnet50_w_norm/XR_ELBOW_resnet50checkpoint_0100.pth.tar",
                     "runs/07-11_11-50-28_XR_SHOULDER_resnet50_w_norm/XR_SHOULDER_resnet50checkpoint_0100.pth.tar",
                     "runs/07-11_12-31-46_XR_HAND_resnet50_w_norm/XR_HAND_resnet50checkpoint_0100.pth.tar"]

In [38]:
for body_part_index in range(3):
    body_part = body_parts[body_part_index]
    cur_weights = weights_normalzie[body_part_index]
    
    train_dataset = MuraDataset("colab_train.pkl", transform=transform, body_part=body_part)
    train_loader = DataLoader(train_dataset, batch_size=2*batch_size, drop_last=False, shuffle=False)
    
    test_dataset = MuraDataset("colab_test.pkl", transform=transform, body_part=body_part)
    test_loader = DataLoader(test_dataset, batch_size=2*batch_size, drop_last=False, shuffle=False)
    
    model = torchvision.models.resnet50(pretrained=False, num_classes=2).to(device)
    
    checkpoint = torch.load(cur_weights, map_location=device)
    # checkpoint = torch.load('pretrain/checkpoint_0100.pth.tar', map_location=device)
    state_dict = checkpoint['state_dict']
    
    for k in list(state_dict.keys()):
    
      if k.startswith('backbone.'):
        if k.startswith('backbone') and not k.startswith('backbone.fc'):
          # remove prefix
          state_dict[k[len("backbone."):]] = state_dict[k]
      del state_dict[k]
    
    log = model.load_state_dict(state_dict, strict=False)
    assert log.missing_keys == ['fc.weight', 'fc.bias']
    
    #freeze all layers but the last fc
    for name, param in model.named_parameters():
        if name not in ['fc.weight', 'fc.bias']:
            param.requires_grad = False
    
    parameters = list(filter(lambda p: p.requires_grad, model.parameters()))
    assert len(parameters) == 2  # fc.weight, fc.bias
    
    trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
    print("Number of trainable parameters:", trainable_params)
    
    optimizer = torch.optim.Adam(model.parameters(), lr=0.0003, weight_decay=0.0008)
    criterion = torch.nn.CrossEntropyLoss().to(device)
    
    def accuracy(output, target, topk=(1,)):
        """Computes the accuracy over the k top predictions for the specified values of k"""
        with torch.no_grad():
            maxk = max(topk)
            batch_size = target.size(0)
    
            _, pred = output.topk(maxk, 1, True, True)
            pred = pred.t()
            correct = pred.eq(target.view(1, -1).expand_as(pred))
    
            res = []
            for k in topk:
                correct_k = correct[:k].reshape(-1).float().sum(0, keepdim=True)
                res.append(correct_k.mul_(100.0 / batch_size))
            return res
    
    now = datetime.now()
    formatted_datetime = "runs/" + now.strftime("%m-%d_%H-%M-%S") 
    log_dir = formatted_datetime + f"finetuned_{body_part}_normalzied"
    writer = SummaryWriter(log_dir=log_dir)
    
    epochs = 100
    for epoch in range(epochs):
      top1_train_accuracy = 0
      all_loss = 0
      model.train()
      optimizer.zero_grad()
      for counter, (x_batch, y_batch) in tqdm(enumerate(train_loader), total=len(train_loader)):
        x_batch = x_batch.to(device)
        y_batch = y_batch.to(device)
    
        logits = model(x_batch)
        loss = criterion(logits, y_batch)
        if counter % 50 == 0:
            writer.add_scalar("loss", loss.item(), counter)
        all_loss += loss
    
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    
      actual_labels = []
      predicted_labels = []
      model.eval()
      for counter, (x_batch, y_batch) in tqdm(enumerate(test_loader), total=len(test_loader)):
        x_batch = x_batch.to(device)
        y_batch = y_batch.to(device)
    
        logits = model(x_batch)
    
        _, predicted = logits.max(1)
        predicted = predicted.detach().cpu().numpy().tolist()
    
        actual_labels.extend(y_batch.detach().cpu().numpy().tolist())
        predicted_labels.extend(predicted)
    
      kappa_score = cohen_kappa_score(actual_labels, predicted_labels)
      roc_score = roc_auc_score(actual_labels, predicted_labels)
      precision = precision_score(actual_labels, predicted_labels)
      recall = recall_score(actual_labels, predicted_labels)
      f1 = f1_score(actual_labels, predicted_labels)
        
      writer.add_scalar("Test/ROC Score", roc_score, epoch)  
      writer.add_scalar("Test/Kappa Score", kappa_score, epoch)
      writer.add_scalar("Test/Precision", precision, epoch)
      writer.add_scalar("Test/Recall", recall, epoch)
      writer.add_scalar("Test/F1-score", f1, epoch)
        
      print(f"Epoch {epoch}\t loss {all_loss/counter}, kappa {kappa_score}, roc: {roc_score}")
    
      if epoch % 20 == 0:
          torch.save(model, f"finetuned/finetuned_{body_part}_epoch_{epoch}")

loaded MURA Dataset (4931, 5)
loaded MURA Dataset (465, 5)


/home/hdamari/venv/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/hdamari/venv/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


Number of trainable parameters: 4098


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████| 15/15 [00:02<00:00,  6.12it/s]


Epoch 0	 loss 7.171902656555176, kappa 0.3221763149343113, roc: 0.6604995374653099


...

Epoch 97	 loss 6.589357376098633, kappa 0.141085202391802, roc: 0.5619594291181007


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████| 15/15 [00:02<00:00,  6.15it/s]


Epoch 98	 loss 6.588289737701416, kappa 0.141085202391802, roc: 0.5619594291181007


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████| 15/15 [00:02<00:00,  6.17it/s]

Epoch 99	 loss 6.587233066558838, kappa 0.141085202391802, roc: 0.5619594291181007


In [31]:
logits.max(1)[1]

tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0], device='cuda:0')

In [25]:
y_batch

tensor([0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 1, 1])

In [65]:
# list(zip(actual_labels, predicted_labels))

In [ ]:
im = x_batch[4].permute(2, 1, 0).cpu().numpy()

In [ ]:
plt.imshow(im)